In [1]:
%store -r __imp

In [2]:
__imp

The libraries have been loaded!


In [3]:
from sklearn.neighbors import NearestNeighbors

In [4]:
rows = 10
X = np.random.randint(10,size=(rows,2))
y = np.random.randint(0,3,rows)
X.shape

(10, 2)

In [5]:
np.unique(y).size

3

In [6]:
k_list = [2,4,5,7,]
knn = NearestNeighbors(n_neighbors=np.unique(y).size)

In [7]:
knn.fit(X, y)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                 radius=1.0)

In [8]:
X, y

(array([[7, 7],
        [8, 0],
        [6, 1],
        [2, 6],
        [2, 4],
        [8, 9],
        [0, 9],
        [2, 6],
        [8, 6],
        [7, 6]]), array([1, 0, 2, 1, 2, 0, 2, 2, 0, 0]))

# test the kneighbors method

In [33]:
# use two instances as an example, but there is only 1 instance in the class
NN_output = knn.kneighbors([[1.5, 2.5]], n_neighbors=max(k_list), return_distance=True)
NN_output

(array([[1.58113883, 3.53553391, 3.53553391, 4.74341649, 6.51920241,
         6.67083203, 6.96419414]]),
 array([[4, 7, 3, 2, 9, 6, 1]], dtype=int64))

In [34]:
# indices of the nearest 4
neighs = NN_output[1][0]
neighs

array([4, 7, 3, 2, 9, 6, 1], dtype=int64)

In [35]:
# distances of the nearest 4
neighs_dist = NN_output[0][0]
neighs_dist

array([1.58113883, 3.53553391, 3.53553391, 4.74341649, 6.51920241,
       6.67083203, 6.96419414])

In [36]:
neighs_y = y[neighs]
n_classes = y.max() + 1
n_classes
neighs_y

array([2, 2, 1, 2, 0, 2, 0])

# 1

    ''' 
        1. Fraction of objects of every class.
           It is basically a KNNСlassifiers predictions.
    '''

In [37]:
# only take the 2 cloest
k = 2
neighs[:k]

array([4, 7], dtype=int64)

In [38]:
# labels of
neighs_y[:k] 

array([2, 2])

In [39]:
labels = np.bincount(neighs_y[:k], minlength=n_classes)
labels

array([0, 0, 2], dtype=int64)

In [40]:
len(k_list), n_classes

(4, 3)

In [41]:
return_list = []
for k in k_list:
    labels = np.bincount(neighs_y[:k],minlength=n_classes)
    feats = labels/labels.sum()
    
    assert len(feats) == n_classes
    return_list += [feats]
return_list

[array([0., 0., 1.]),
 array([0.  , 0.25, 0.75]),
 array([0.2, 0.2, 0.6]),
 array([0.28571429, 0.14285714, 0.57142857])]

# 2

    '''
        2. Same label streak: the largest number N, 
           such that N nearest neighbors have the same label.
    '''

In [42]:
# N label streak
for a in [np.array([2,2,2,2,0,1,1,0]), np.array([1,2,2,2,0,1,1,0]), np.array([0,0,np.nan,2,0,1,1,0]), np.array([0,0,0,0,0,0,0])]:
    print(a, np.where(np.unique(a).size==1, a.size, np.argmin(a[:-1]==a[1:]) + 1))

[2 2 2 2 0 1 1 0] 4
[1 2 2 2 0 1 1 0] 1
[ 0.  0. nan  2.  0.  1.  1.  0.] 2
[0 0 0 0 0 0 0] 7


In [43]:
neighs_y
np.where(np.unique(neighs_y).size==1, neighs_y.size, np.argmin(neighs_y[:-1]==neighs_y[1:]) + 1)

array(2, dtype=int64)

In [70]:
# test
feats =  np.where(np.unique(neighs_y).size==1, neighs_y.size, np.argmin(neighs_y[:-1]==neighs_y[1:]) + 1)

assert feats.size == 1
return_list += [feats]
return_list

# 3

    '''
        3. Minimum distance to objects of each class
           Find the first instance of a class and take its distance as features.

           If there are no neighboring objects of some classes, 
           Then set distance to that class to be 999.
    '''

In [45]:
neighs_y
neighs_dist
c = 2
dis_array = neighs_dist[np.where(neighs_y==c)]
dis_array
dis = 999 if dis_array.size==0 else dis_array[0]
dis

1.5811388300841898

In [46]:
feats = []
for c in range(n_classes):
    dis_array = neighs_dist[np.where(neighs_y==c)]
    dis = 999 if dis_array.size==0 else dis_array[0]
    feats.append(dis)

# feats = np.array(feats).reshape(-1,1)
assert len(feats) == n_classes
feats

[6.519202405202649, 3.5355339059327378, 1.5811388300841898]

In [47]:
return_list += [feats]
return_list

[array([0., 0., 1.]),
 array([0.  , 0.25, 0.75]),
 array([0.2, 0.2, 0.6]),
 array([0.28571429, 0.14285714, 0.57142857]),
 array([2], dtype=int64),
 [6.519202405202649, 3.5355339059327378, 1.5811388300841898]]

# 4

    '''
        4. Minimum *normalized* distance to objects of each class
           As 3. but we normalize (divide) the distances
           by the distance to the closest neighbor.

           If there are no neighboring objects of some classes, 
           Then set distance to that class to be 999.

           Do not forget to add self.eps to denominator.
    '''

In [48]:
feats = []
eps=1e-6
for c in range(n_classes):
    dis_array = neighs_dist[np.where(neighs_y==c)]
    dis = 999 if dis_array.size==0 else dis_array[0] / (neighs_dist[0] + eps)
    feats.append(dis)

# feats = np.array(feats).reshape(-1,1)
assert len(feats) == n_classes
feats

[4.123103017938348, 2.236066563287122, 0.9999993675448681]

In [49]:
return_list += [feats]
return_list

[array([0., 0., 1.]),
 array([0.  , 0.25, 0.75]),
 array([0.2, 0.2, 0.6]),
 array([0.28571429, 0.14285714, 0.57142857]),
 array([2], dtype=int64),
 [6.519202405202649, 3.5355339059327378, 1.5811388300841898],
 [4.123103017938348, 2.236066563287122, 0.9999993675448681]]

# 5

    '''
        5. 
           5.1 Distance to Kth neighbor
               Think of this as of quantiles of a distribution
           5.2 Distance to Kth neighbor normalized by 
               distance to the first neighbor

           feat_51, feat_52 are answers to 5.1. and 5.2.
           should be scalars

           Do not forget to add self.eps to denominator.
    '''

In [50]:
k_list,neighs_dist

([2, 4, 5, 7],
 array([1.58113883, 3.53553391, 3.53553391, 4.74341649, 6.51920241,
        6.67083203, 6.96419414]))

In [51]:
for k in k_list:
    feat_51 = neighs_dist[k-1]
    feat_52 = neighs_dist[k-1] / (neighs_dist[0] + eps)
#     print(feat_51, feat_52)
    return_list += [[feat_51, feat_52]]
return_list

[array([0., 0., 1.]),
 array([0.  , 0.25, 0.75]),
 array([0.2, 0.2, 0.6]),
 array([0.28571429, 0.14285714, 0.57142857]),
 array([2], dtype=int64),
 [6.519202405202649, 3.5355339059327378, 1.5811388300841898],
 [4.123103017938348, 2.236066563287122, 0.9999993675448681],
 [3.5355339059327378, 2.236066563287122],
 [4.743416490252569, 2.999998102634604],
 [6.519202405202649, 4.123103017938348],
 [6.96419413859206, 4.404540323433155]]

# 6

    '''
    6. Mean distance to neighbors of each class for each K from `k_list` 
       For each class select the neighbors of that class among K nearest neighbors 
       and compute the average distance to those objects

       If there are no objects of a certain class among K neighbors, set mean distance to 999

       You can use `np.bincount` with appropriate weights
       Don't forget, that if you divide by something, 
       You need to add `self.eps` to denominator.
        '''


In [52]:
k = 2
neighs_y[:k], neighs_dist[:k]
np.bincount(neighs_y[:k])
mean_dist = np.bincount(neighs_y[:k], neighs_dist[:k]) / (np.bincount(neighs_y[:k]) + eps)
mean_dist
feats = np.where(mean_dist==0,999,mean_dist)
feats

array([999.        , 999.        ,   2.55833509])

In [53]:
for k in k_list:
    mean_dist = np.bincount(neighs_y[:k], weights=neighs_dist[:k], minlength=n_classes) \
        / (np.bincount(neighs_y[:k], minlength=n_classes) + eps)
    feats = np.where(mean_dist==0, 999, mean_dist)
    
    assert len(feats) == n_classes
    return_list += [feats]

In [54]:
return_list

[array([0., 0., 1.]),
 array([0.  , 0.25, 0.75]),
 array([0.2, 0.2, 0.6]),
 array([0.28571429, 0.14285714, 0.57142857]),
 array([2], dtype=int64),
 [6.519202405202649, 3.5355339059327378, 1.5811388300841898],
 [4.123103017938348, 2.236066563287122, 0.9999993675448681],
 [3.5355339059327378, 2.236066563287122],
 [4.743416490252569, 2.999998102634604],
 [6.519202405202649, 4.123103017938348],
 [6.96419413859206, 4.404540323433155],
 array([999.        , 999.        ,   2.55833509]),
 array([999.        ,   3.53553037,   3.28669531]),
 array([6.51919589, 3.53553037, 3.28669531]),
 array([6.7416949 , 3.53553037, 4.13272928])]

# stack

In [56]:
knn_feats = np.hstack(return_list)
knn_feats.shape, knn_feats

((39,), array([0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        2.50000000e-01, 7.50000000e-01, 2.00000000e-01, 2.00000000e-01,
        6.00000000e-01, 2.85714286e-01, 1.42857143e-01, 5.71428571e-01,
        2.00000000e+00, 6.51920241e+00, 3.53553391e+00, 1.58113883e+00,
        4.12310302e+00, 2.23606656e+00, 9.99999368e-01, 3.53553391e+00,
        2.23606656e+00, 4.74341649e+00, 2.99999810e+00, 6.51920241e+00,
        4.12310302e+00, 6.96419414e+00, 4.40454032e+00, 9.99000000e+02,
        9.99000000e+02, 2.55833509e+00, 9.99000000e+02, 3.53553037e+00,
        3.28669531e+00, 6.51919589e+00, 3.53553037e+00, 3.28669531e+00,
        6.74169490e+00, 3.53553037e+00, 4.13272928e+00]))